<a href="https://colab.research.google.com/github/19k41a05g6/NNDL/blob/main/nndl_ass_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Load Data in kW.csv')
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


In [4]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7]) #Since x is 24 * 7 hours before y
    y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.head()

,Previous Week Load,Present Week Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [6]:
maxX = np.max(data['Previous Week Load'])
minX = np.min(data['Previous Week Load'])
maxY = np.max(data['Present Week Load'])
minY = np.min(data['Present Week Load'])

data['Previous Week Load'] = (data['Previous Week Load'] - minX) / (maxX - minX)
data['Present Week Load'] = (data['Present Week Load'] - minY) / (maxY - minY)

In [7]:
x = data['Previous Week Load']
y = data['Present Week Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [8]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6
for _ in range(epochs):
    for i in range(trainX.shape[0]):
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)
print(f'm = {m}\nc = {c}')

m = 0.4857356394633998
c = 0.23135623893509416


In [9]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 737.0270667663004
MSE : 891424.2758690235
RMSE : 944.1526761435481
Testing Error :
MAE : 787.1168535670776
MSE : 968114.9814551049
RMSE : 983.9283416261089


In [10]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,5086.98792,5060.576513
1,6378.60960,5731.523567
2,7647.78456,6182.155125
3,6165.05400,5656.064752
4,5918.45184,5501.813600
...,...,...
1809,5020.11540,5866.682729
1810,4048.82712,5524.163349
1811,5674.03200,5578.181557
1812,5621.65632,6995.534480


In [11]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,7609.43808,6756.051233
1,4187.24856,5748.797271
2,6738.69240,6181.549030
3,5640.67368,6206.095873
4,6550.07760,5442.416300
...,...,...
197,5761.79244,6598.845368
198,6955.67736,6087.149749
199,8607.69360,6021.085405
200,6532.93080,6219.505723


In [12]:
x = float(input('Enter the load at same hour in previous week: '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at same hour in previous week: 96
Predicted load at present hour : 3045.300239988528
